In [31]:
import pandas as pd
#import pandavro as pda
import lxml.html as lh
from urllib.request import urlopen, Request
import requests
pd.__version__


'1.0.1'

In [47]:
url = "http://www.brewunited.com/yeast_database.php"
print(url)

http://www.brewunited.com/yeast_database.php


In [61]:


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(url=url, headers=headers) 
page = requests.get(url)

print(page) 
doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')
print(tr_elements)


#Check the length of the first 50 rows
[len(T) for T in tr_elements[:50]]


<Response [200]>
[<Element tr at 0x18930b8a0e8>, <Element tr at 0x18930b8aef8>, <Element tr at 0x18930b8af48>, <Element tr at 0x18930b8ae58>, <Element tr at 0x18930b9e2c8>, <Element tr at 0x18930b263b8>, <Element tr at 0x1893076ae08>, <Element tr at 0x18930b0ad68>, <Element tr at 0x18930b0adb8>, <Element tr at 0x1893074a7c8>, <Element tr at 0x18930751778>, <Element tr at 0x18930b29278>, <Element tr at 0x18930b271d8>, <Element tr at 0x18930b27368>, <Element tr at 0x18930b27408>, <Element tr at 0x189308fa9a8>, <Element tr at 0x189308faa48>, <Element tr at 0x1892de87098>, <Element tr at 0x18930b1ef98>, <Element tr at 0x18930b868b8>, <Element tr at 0x18930b86f98>, <Element tr at 0x18930b86048>, <Element tr at 0x18930b869a8>, <Element tr at 0x18930743868>, <Element tr at 0x18930bf4db8>, <Element tr at 0x18930bf55e8>, <Element tr at 0x18930bf5db8>, <Element tr at 0x18930c07638>, <Element tr at 0x18930c07ae8>, <Element tr at 0x18930bf62c8>, <Element tr at 0x18930bf6098>, <Element tr at 0x1893

[8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

In [62]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Name"
2:"Lab"
3:"Type"
4:"Form"
5:"Temp"
6:"Attenuation"
7:"Flocculation"
8:"Notes"


In [63]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=8:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [64]:
[len(C) for (title,C) in col]
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head(20) 

,Name,Lab,Type,Form,Temp,Attenuation,Flocculation,Notes
0,Frankenyeast,Various,Ale,Liquid,62°F - 75°F,75.0%,Low,"A blend of twenty-five yeast strains, most of ..."
1,Nottingham Ale Yeast,White Labs,Ale,Liquid,32°F - 32°F,77.5%,,
2,Array American Hefeweizen Ale Yeast WLP320,White Labs,Ale,Liquid,65°F - 69°F,76.0%,Low,
3,WLP001 California Ale,White Labs,Ale,Liquid,68°F - 73°F,76.5%,High,"Very clean flavor, balance and stability. Acce..."
4,WLP002 English Ale,White Labs,Ale,Liquid,65°F - 68°F,66.5%,Very High,Classic ESB strain best for English style mild...
5,WLP003 German Ale II,White Labs,Ale,Liquid,65°F - 70°F,76.5%,Medium,Strong sulfer component will reduce with aging...
6,WLP004 Irish Ale Yeast,White Labs,Ale,Liquid,65°F - 68°F,71.5%,Medium,Excellent for Irish Stouts. Produces slight hi...
7,WLP005 British Ale,White Labs,Ale,Liquid,67°F - 74°F,70.5%,High,This yeast has higher attenuation than the Whi...
8,WLP006 Bedford British Ale,White Labs,Ale,Liquid,65°F - 70°F,76.0%,High,High attenuation. Ferments dry with high flocc...
9,WLP007 Dry English Ale,White Labs,Ale,Liquid,65°F - 70°F,75.0%,High,"Clean, Highly flocculant, and highly attentive..."


In [65]:
df.count()

Name            253
Lab             253
Type            253
Form            253
Temp            253
Attenuation     253
Flocculation    253
Notes           253
dtype: int64

In [66]:
##cleaning Duplicates and rubish
df = df.drop_duplicates() 
df.count()


#df = df[df['Hop'] != 'Hop']
#df = df[df['Hop'] != '0']

Name            225
Lab             225
Type            225
Form            225
Temp            225
Attenuation     225
Flocculation    225
Notes           225
dtype: int64

In [67]:
df.head(50)
df.count()

Name            225
Lab             225
Type            225
Form            225
Temp            225
Attenuation     225
Flocculation    225
Notes           225
dtype: int64

In [68]:

df = df[df['Name'] != 'Name']

df.count()

Name            224
Lab             224
Type            224
Form            224
Temp            224
Attenuation     224
Flocculation    224
Notes           224
dtype: int64

In [69]:
df.head(60)

,Name,Lab,Type,Form,Temp,Attenuation,Flocculation,Notes
0,Frankenyeast,Various,Ale,Liquid,62°F - 75°F,75.0%,Low,"A blend of twenty-five yeast strains, most of ..."
1,Nottingham Ale Yeast,White Labs,Ale,Liquid,32°F - 32°F,77.5%,,
2,Array American Hefeweizen Ale Yeast WLP320,White Labs,Ale,Liquid,65°F - 69°F,76.0%,Low,
3,WLP001 California Ale,White Labs,Ale,Liquid,68°F - 73°F,76.5%,High,"Very clean flavor, balance and stability. Acce..."
4,WLP002 English Ale,White Labs,Ale,Liquid,65°F - 68°F,66.5%,Very High,Classic ESB strain best for English style mild...
5,WLP003 German Ale II,White Labs,Ale,Liquid,65°F - 70°F,76.5%,Medium,Strong sulfer component will reduce with aging...
6,WLP004 Irish Ale Yeast,White Labs,Ale,Liquid,65°F - 68°F,71.5%,Medium,Excellent for Irish Stouts. Produces slight hi...
7,WLP005 British Ale,White Labs,Ale,Liquid,67°F - 74°F,70.5%,High,This yeast has higher attenuation than the Whi...
8,WLP006 Bedford British Ale,White Labs,Ale,Liquid,65°F - 70°F,76.0%,High,High attenuation. Ferments dry with high flocc...
9,WLP007 Dry English Ale,White Labs,Ale,Liquid,65°F - 70°F,75.0%,High,"Clean, Highly flocculant, and highly attentive..."


In [70]:
dup = df[df.duplicated(['Name'])].sort_values(by=['Name'])
dup.head(65)

,Name,Lab,Type,Form,Temp,Attenuation,Flocculation,Notes
195,Belle Saison,Danstar,Ale,Dry,63°F - 77°F,80.0%,Low,Best for: Saison


In [71]:
df[df['Name'] == 'Belle Saison']

,Name,Lab,Type,Form,Temp,Attenuation,Flocculation,Notes
194,Belle Saison,Danstar,Ale,Liquid,64°F - 90°F,82.5%,Low,Best for: Saisons and belgians
195,Belle Saison,Danstar,Ale,Dry,63°F - 77°F,80.0%,Low,Best for: Saison


In [72]:
df = df.drop([194])


In [73]:
df = df.sort_values(by=['Name'])

In [81]:
df.to_json (r'C:\Users\bceolincamar\Documents\GitHub\homebrew\Export_Yeast.json', orient='records')

In [40]:
df.rename(columns={ df.columns[6]: "MaxPercent" }, inplace = True)
df.rename(columns={ df.columns[2]: "Mash" }, inplace = True)

In [75]:
df.head(60)

,Name,Lab,Type,Form,Temp,Attenuation,Flocculation,Notes
190,- Cooper Ale,Coopers,Ale,Dry,62°F - 72°F,75.0%,Medium,General purpose dry ale yeast with a very good...
202,- Diamond Lager,Lallemand,Lager,Dry,50°F - 59°F,73.0%,Medium,Diamond Lager yeast originated in Germany and ...
199,- Doric Ale,Doric,Ale,Dry,62°F - 72°F,75.0%,Medium,Good reputation. Reliable clean finish for a d...
200,- Edme Ale Yeast,Edme,Ale,Dry,62°F - 72°F,75.0%,Medium,Quick starting dry yeast with a good reputatio...
196,- London,Danstar,Ale,Dry,64°F - 70°F,72.0%,Medium,"Produces a clean, well balanced ale. Medium at..."
197,- Manchester,Danstar,Ale,Dry,64°F - 70°F,72.0%,Medium,Old english style ale yeast that produces a co...
212,- Munton Fison Ale,Munton-Fison,Ale,Dry,62°F - 72°F,75.0%,Medium,Quick starting dry yeast. Produces some fuity ...
204,- Munuch,Lallemand,Wheat,Dry,65°F - 71°F,70.0%,Medium,Danstar Munich Wheat Beer yeast originated in ...
198,- Nottingham,Danstar,Ale,Dry,57°F - 70°F,75.0%,High,Dry strain is highly flocculant and has high a...
205,- Nottingham Yeast,Lallemand,Ale,Dry,62°F - 72°F,75.0%,Very High,High flocculation - settles quickly. Very good...


In [78]:
#df = df['Name'].str.replace('- ','')
#df.head(60)

df = df.replace('- ', '', regex=True)
df.head(60)

,Name,Lab,Type,Form,Temp,Attenuation,Flocculation,Notes
190,Cooper Ale,Coopers,Ale,Dry,62°F 72°F,75.0%,Medium,General purpose dry ale yeast with a very good...
202,Diamond Lager,Lallemand,Lager,Dry,50°F 59°F,73.0%,Medium,Diamond Lager yeast originated in Germany and ...
199,Doric Ale,Doric,Ale,Dry,62°F 72°F,75.0%,Medium,Good reputation. Reliable clean finish for a d...
200,Edme Ale Yeast,Edme,Ale,Dry,62°F 72°F,75.0%,Medium,Quick starting dry yeast with a good reputatio...
196,London,Danstar,Ale,Dry,64°F 70°F,72.0%,Medium,"Produces a clean, well balanced ale. Medium at..."
197,Manchester,Danstar,Ale,Dry,64°F 70°F,72.0%,Medium,Old english style ale yeast that produces a co...
212,Munton Fison Ale,Munton-Fison,Ale,Dry,62°F 72°F,75.0%,Medium,Quick starting dry yeast. Produces some fuity ...
204,Munuch,Lallemand,Wheat,Dry,65°F 71°F,70.0%,Medium,Danstar Munich Wheat Beer yeast originated in ...
198,Nottingham,Danstar,Ale,Dry,57°F 70°F,75.0%,High,Dry strain is highly flocculant and has high a...
205,Nottingham Yeast,Lallemand,Ale,Dry,62°F 72°F,75.0%,Very High,High flocculation settles quickly. Very good r...


In [79]:
df = df.sort_values(by=['Name'])

In [80]:
df.head(60)

,Name,Lab,Type,Form,Temp,Attenuation,Flocculation,Notes
99,1007 German Ale,Wyeast,Ale,Liquid,55°F 66°F,75.0%,Low,"Crisp, dry finish with a mild flavor.Best for:..."
100,1010 American Wheat 1010,Wyeast,Wheat,Liquid,58°F 74°F,76.0%,Low,
101,1010 American Wheat Ale,Wyeast,Ale,Liquid,58°F 74°F,76.0%,Low,"Dry, Crisp, tart beer in the American Hefeweiz..."
102,1026 British Cask Ale,Wyeast,Ale,Liquid,63°F 72°F,75.5%,Medium,A great choice for any cask conditioned Britis...
103,1028 London Ale Yeast,Wyeast,Ale,Liquid,60°F 72°F,75.0%,Medium,"Dry finish, bold, rich flavor, some fruit prof..."
104,1056 American Ale,Wyeast,Ale,Liquid,60°F 72°F,75.0%,Medium,"Soft, smooth, clean finish. Very well balanced..."
105,1084 Irish Ale,Wyeast,Ale,Liquid,62°F 72°F,73.0%,Medium,"Dry diacetyl, fruity flavor characteristic of ..."
106,1087 Wyeast Ale Blend,Wyeast,Ale,Liquid,64°F 72°F,73.0%,Medium,Blend of ale strains designed to provide quick...
108,1098 British Ale Yeast,Wyeast,Ale,Liquid,64°F 72°F,74.0%,Medium,"Fruity, tart, dry crisp finish. Very well bala..."
109,1099 Whitbread Ale,Wyeast,Ale,Liquid,64°F 75°F,70.0%,High,Slightly more fruity and malty than Wyeast's B...
